In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
import sklearn
import statsmodels
import arch

In [3]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np

In [4]:
# Helper function: Data acquisition
def load_data(stock_file, company_file, index_file):
    stocks = pd.read_csv(stock_file)
    companies = pd.read_csv(company_file)
    index = pd.read_csv(index_file)
    return stocks, companies, index

In [5]:
def convert_dates(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    return df

In [6]:
# Helper function: Data cleaning
def clean_data(stocks):
    # Drop rows with missing values or interpolate them
    # stocks.interpolate(method='linear', inplace=True)
    
    # Interpolate only numeric columns
    numeric_cols = stocks.select_dtypes(include=['float64', 'int64']).columns
    stocks[numeric_cols] = stocks[numeric_cols].interpolate(method='linear')
    return stocks

In [7]:
# Helper function: Normalize numerical features
def normalize_features(stocks, columns_to_scale):
    scaler = StandardScaler()
    stocks[columns_to_scale] = scaler.fit_transform(stocks[columns_to_scale])
    return stocks

In [8]:
# Helper function: Feature engineering
def add_features(stocks):
    # Calculate daily returns
    stocks['daily_return'] = stocks['Close'].pct_change()

    # Add moving averages
    stocks['MA_10'] = stocks['Close'].rolling(window=10).mean()
    stocks['MA_50'] = stocks['Close'].rolling(window=50).mean()

    # Calculate rolling volatility
    stocks['volatility'] = stocks['daily_return'].rolling(window=10).std()

    return stocks

In [9]:
def optimize_memory(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df


In [10]:
# Main execution
# File paths (adjust as necessary)
stock_file = "sp500_stocks.csv"
company_file = "sp500_companies.csv"
index_file = "sp500_index.csv"

# Step 1: Load datasets
print("\nLoading datasets...")
stocks, companies, index = load_data(stock_file, company_file, index_file)



Loading datasets...


In [11]:

# Step 2: Verify data structure
print("Stocks Data Overview:")
print(stocks.info())
print("\nCompanies Data Overview:")
print(companies.info())
print("\nIndex Data Overview:")
print(index.info())


Stocks Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883735 entries, 0 to 1883734
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Date       object 
 1   Symbol     object 
 2   Adj Close  float64
 3   Close      float64
 4   High       float64
 5   Low        float64
 6   Open       float64
 7   Volume     float64
dtypes: float64(6), object(2)
memory usage: 115.0+ MB
None

Companies Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exchange             503 non-null    object 
 1   Symbol               503 non-null    object 
 2   Shortname            503 non-null    object 
 3   Longname             503 non-null    object 
 4   Sector               503 non-null    object 
 5   Industry             503 non-null    object 
 6   Currentprice         503 non-nu

In [12]:

# Step 3: Data cleaning
print("\nCleaning stocks data...")
stocks = clean_data(stocks)




Cleaning stocks data...


In [13]:
# Step 4: Convert Date columns to datetime format
print("\nConverting date columns to datetime...")
stocks = convert_dates(stocks, 'Date')
index = convert_dates(index, 'Date')




Converting date columns to datetime...


In [14]:
# Step 5: Optimize memory usage for stocks
print("\nOptimizing memory usage...")
stocks = optimize_memory(stocks)




Optimizing memory usage...


In [15]:
# Step 6: Join stocks data with company metadata
# stocks = stocks.merge(companies, left_on='Symbol', right_on='Symbol', how='left')
print("\nJoining stocks data with company metadata...")
stocks = stocks.merge(companies[['Symbol', 'Sector', 'Industry']], on='Symbol', how='left')




Joining stocks data with company metadata...


In [16]:
# Step 7: Normalize numerical features
print("\nNormalizing numerical features...")
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume']
stocks = normalize_features(stocks, columns_to_scale)





Normalizing numerical features...


In [17]:
# Step 8: Feature engineering
print("\nAdding new features...")
stocks = add_features(stocks)




Adding new features...


In [18]:
# Save processed data for future use
processed_file = "processed_sp500_data.csv"
print(f"\nSaving processed data to {processed_file}...")
stocks.to_csv(processed_file, index=False)

print("Data preprocessing and feature engineering completed.")


Saving processed data to processed_sp500_data.csv...
Data preprocessing and feature engineering completed.


In [19]:
# Verify the Processed Data File
import pandas as pd

# Load the processed data
processed_file = "processed_sp500_data.csv"
processed_stocks = pd.read_csv(processed_file)

# Check basic information about the data
print("Processed Data Overview:")
print(processed_stocks.info())

# Display the first few rows of the dataset
print("\nFirst 5 Rows of Processed Data:")
print(processed_stocks.head())


Processed Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883735 entries, 0 to 1883734
Data columns (total 14 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Date          object 
 1   Symbol        object 
 2   Adj Close     float64
 3   Close         float64
 4   High          float64
 5   Low           float64
 6   Open          float64
 7   Volume        float64
 8   Sector        object 
 9   Industry      object 
 10  daily_return  float64
 11  MA_10         float64
 12  MA_50         float64
 13  volatility    float64
dtypes: float64(10), object(4)
memory usage: 201.2+ MB
None

First 5 Rows of Processed Data:
         Date Symbol  Adj Close     Close      High       Low      Open  \
0  2010-01-04    MMM  43.783863 -0.254543 -0.256966 -0.251168 -0.254309   
1  2010-01-05    MMM  43.509640 -0.257461 -0.258192 -0.256681 -0.255951   
2  2010-01-06    MMM  44.126670 -0.250895 -0.250554 -0.246394 -0.249837   
3  2010-01-07    MMM  44.158306 -0.25

In [20]:
# Check for missing values in each column
missing_values = processed_stocks.isnull().sum()
print("\nMissing Values in Processed Data:")
print(missing_values[missing_values > 0])  # Display only columns with missing values

if missing_values.sum() == 0:
    print("\nNo missing values found in the dataset.")
else:
    print("\nSome columns still contain missing values. Consider handling them.")



Missing Values in Processed Data:
daily_return     1
MA_10            9
MA_50           49
volatility      10
dtype: int64

Some columns still contain missing values. Consider handling them.


In [21]:
# Check if feature engineering columns exist
# This block ensures that the engineered features (daily_return, MA_10, MA_50, volatility) were added correctly.

engineered_columns = ['daily_return', 'MA_10', 'MA_50', 'volatility']
print("\nFeature Engineering Check:")
for col in engineered_columns:
    if col in processed_stocks.columns:
        print(f"'{col}' exists in the dataset. Sample values:")
        print(processed_stocks[col].head())
    else:
        print(f"'{col}' is missing from the dataset.")



Feature Engineering Check:
'daily_return' exists in the dataset. Sample values:
0         NaN
1    0.011465
2   -0.025504
3   -0.001342
4   -0.013216
Name: daily_return, dtype: float64
'MA_10' exists in the dataset. Sample values:
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: MA_10, dtype: float64
'MA_50' exists in the dataset. Sample values:
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: MA_50, dtype: float64
'volatility' exists in the dataset. Sample values:
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: volatility, dtype: float64


In [22]:
import numpy as np

# Check for NaN, infinity, or invalid values in numerical columns
numerical_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'daily_return', 'MA_10', 'MA_50', 'volatility']
invalid_values = processed_stocks[numerical_cols].apply(lambda x: (~np.isfinite(x)).sum())
print("\nInvalid Values Check:")
print(invalid_values[invalid_values > 0])  # Display only columns with invalid values

if invalid_values.sum() == 0:
    print("\nNo invalid values found in numerical columns.")
else:
    print("\nSome numerical columns contain invalid values.")

# processed_stocks.dropna(inplace=True)




Invalid Values Check:
daily_return     1
MA_10            9
MA_50           49
volatility      10
dtype: int64

Some numerical columns contain invalid values.


In [23]:
# Verify Sector and Industry columns
print("\nMetadata Columns Check:")
metadata_columns = ['Sector', 'Industry']
for col in metadata_columns:
    if col in processed_stocks.columns:
        print(f"'{col}' exists in the dataset. Sample values:")
        print(processed_stocks[col].value_counts().head())
    else:
        print(f"'{col}' is missing from the dataset.")



Metadata Columns Check:
'Sector' exists in the dataset. Sample values:
Sector
Technology            307090
Industrials           262150
Financial Services    250915
Healthcare            235935
Consumer Cyclical     205975
Name: count, dtype: int64
'Industry' exists in the dataset. Sample values:
Industry
Utilities - Regulated Electric    86135
Specialty Industrial Machinery    59920
Software - Application            56175
Semiconductors                    52430
Software - Infrastructure         52430
Name: count, dtype: int64


In [24]:
# # Ensure 'Date' column is in datetime format
# processed_stocks['Date'] = pd.to_datetime(processed_stocks['Date'])

# # Filter rows where the year is 2023
# stocks_2023 = processed_stocks[processed_stocks['Date'].dt.year == 2023]

# # Display the first few rows of the filtered dataset
# print("Stocks Data for 2023:")
# print(stocks_2023.info())
# print(stocks_2023.tail(n=50))


In [25]:
processed_stocks

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Sector,Industry,daily_return,MA_10,MA_50,volatility
0,2010-01-04,MMM,43.783863,-0.254543,-0.256966,-0.251168,-0.254309,-0.175490,Industrials,Conglomerates,NaN,NaN,NaN,NaN
1,2010-01-05,MMM,43.509640,-0.257461,-0.258192,-0.256681,-0.255951,-0.191073,Industrials,Conglomerates,0.011465,NaN,NaN,NaN
2,2010-01-06,MMM,44.126670,-0.250895,-0.250554,-0.246394,-0.249837,0.000770,Industrials,Conglomerates,-0.025504,NaN,NaN,NaN
3,2010-01-07,MMM,44.158306,-0.250558,-0.255237,-0.254294,-0.253007,-0.062484,Industrials,Conglomerates,-0.001342,NaN,NaN,NaN
4,2010-01-08,MMM,44.469480,-0.247247,-0.252115,-0.247588,-0.250913,-0.146802,Industrials,Conglomerates,-0.013216,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883730,2024-11-12,ZTS,113.020000,0.038151,0.042718,0.044635,0.038344,-0.291565,Healthcare,Drug Manufacturers - Specialty & Generic,0.000000,0.038151,0.038151,0.0
1883731,2024-11-13,ZTS,113.020000,0.038151,0.042718,0.044635,0.038344,-0.291565,Healthcare,Drug Manufacturers - Specialty & Generic,0.000000,0.038151,0.038151,0.0
1883732,2024-11-14,ZTS,113.020000,0.038151,0.042718,0.044635,0.038344,-0.291565,Healthcare,Drug Manufacturers - Specialty & Generic,0.000000,0.038151,0.038151,0.0
1883733,2024-11-15,ZTS,113.020000,0.038151,0.042718,0.044635,0.038344,-0.291565,Healthcare,Drug Manufacturers - Specialty & Generic,0.000000,0.038151,0.038151,0.0
